In [19]:
from transformers import Wav2Vec2FeatureExtractor, HubertModel
import torchaudio
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import os
import pickle
import requests
import sounddevice as sd
from scipy.io.wavfile import write
import threading

In [2]:
# 1. HuBERT 특성 추출기 정의
class HuBERTFeatureExtractor:
    def __init__(self, model_name="facebook/hubert-base-ls960"):
        # HuBERT 모델과 프로세서 초기화
        self.processor = Wav2Vec2FeatureExtractor.from_pretrained(model_name)
        self.model = HubertModel.from_pretrained(model_name)
        self.model.eval()  # 평가 모드로 설정

    def load_audio(self, audio_file):
        # 오디오 파일 로드
        waveform, sample_rate = torchaudio.load(audio_file, format="wav")
        
        return waveform, sample_rate

    def preprocess_audio(self, waveform, sample_rate, target_sample_rate=44100, max_length=10):
        # 모노로 변환
        if waveform.size(0) > 1:
            waveform = waveform.mean(dim=0, keepdim=True)
        
        # 샘플링 레이트 변환
        if sample_rate != target_sample_rate:
            resampler = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=target_sample_rate)
            waveform = resampler(waveform)
            
        max_samples = target_sample_rate * max_length
        if waveform.size(1) > max_samples:
            waveform = waveform[:, :max_samples]
            
        return waveform

    def extract_features(self, audio_file):
        # 오디오 로드 및 전처리
        waveform, sample_rate = self.load_audio(audio_file)
        waveform = self.preprocess_audio(waveform, sample_rate)
        
        # 입력 차원 확인 및 조정
        if waveform.dim() == 1:
            waveform = waveform.unsqueeze(0)
        elif waveform.dim() == 2:
            if waveform.size(0) > 1:
                waveform = waveform.mean(dim=0, keepdim=True)
        elif waveform.dim() == 3:
            waveform = waveform.squeeze(0)
            if waveform.size(0) > 1:
                waveform = waveform.mean(dim=0, keepdim=True)
        
        # 특성 추출
        inputs = self.processor(waveform, sampling_rate=16000, return_tensors="pt", padding=True)
        input_values = inputs.input_values
        
        # 불필요한 차원 제거
        input_values = input_values.squeeze(1)  # (batch_size=1, sequence_length)
        
        with torch.no_grad():
            outputs = self.model(input_values)
            
        features = outputs.last_hidden_state
        
        return features

In [3]:
# 2. 데이터셋 클래스 정의
class EmotionDataset:
    def __init__(self, file_paths, feature_extractor):
        self.file_paths = file_paths
        self.feature_extractor = feature_extractor

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        audio_path = self.file_paths[idx]
        features = self.feature_extractor.extract_features(audio_path)
        features = features.squeeze(0)  # (sequence_length, hidden_size)
        return features

In [4]:
# 3. Transformer 모델 정의
class EmotionTransformer(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(EmotionTransformer, self).__init__()
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=input_dim, nhead=16),
            num_layers=8
        )
        self.fc = nn.Linear(input_dim, num_classes)

    def forward(self, x, src_key_padding_mask=None):
        # Transformer를 통해 특성 추출
        x = self.transformer(x, src_key_padding_mask=src_key_padding_mask)
        x = x[-1]  # Sequence의 평균을 사용
        output = self.fc(x)
        return output

In [5]:
def predict_emotion(dataloader, model, label_encoder):
    device = torch.device("mps")
    model = model.to(device)

    predictions = []
    with torch.no_grad():
        for features in dataloader:
            features = features.to(device)
            outputs = model(features)
            _, predicted = torch.max(outputs, 1)
            predictions.extend(predicted.cpu().numpy())

    # 예측값을 레이블로 변환
    decoded_predictions = label_encoder.inverse_transform(predictions)
    return decoded_predictions

In [6]:
def collate_fn(batch):
    features = [item for item in batch]
    features = torch.nn.utils.rnn.pad_sequence(features, batch_first=True)  # [batch_size, sequence_length, feature_dim]
    return features

In [7]:
# TMDB API 키
TMDB_API_KEY = "df9a0caaf2a07ee6babd7024a6accaf8"
    
EMOTION_TO_GENRE = {
    '기쁨': 35,  # Comedy
    '슬픔': 18,  # Drama
    '분노': 53,  # Thriller
    '불안': 27,  # Horror
    '상처': 80,  # Crime
    '당황': 28,  # Action
    '중립': 10751,  # Family
}

def get_recommendations(emotion, result_num=10, api_key=TMDB_API_KEY):
    # 감정 매핑 확인
    genre_id = EMOTION_TO_GENRE.get(emotion)
    if not genre_id:
        return f"'{emotion}'에 해당하는 추천 장르가 없습니다. 감정을 다시 입력해주세요."

    # TMDB Discover API 호출
    url = f"https://api.themoviedb.org/3/discover/movie"
    params = {
        "api_key": api_key,
        "with_genres": genre_id,
        "sort_by": "popularity.desc",  # 인기 순으로 정렬
        "language": "ko-KR",          # 한국어 결과
        "vote_average.gte": 7.0,      # 평점 7 이상
    }

    response = requests.get(url, params=params)
    if response.status_code != 200:
        return f"TMDB API 호출 실패: {response.status_code}"

    data = response.json()
    results = data.get("results", [])

    if not results:
        return f"'{emotion}'에 맞는 추천 콘텐츠를 찾을 수 없습니다."

    # 추천 콘텐츠 추출
    recommendations = []
    for movie in results[:result_num]:  # 상위 N개만 추출
        recommendations.append({
            "title": movie.get("title"),
            "overview": movie.get("overview"),
            "vote_average": movie.get("vote_average"),
            "release_date": movie.get("release_date"),
        })

    return recommendations

In [11]:
model = torch.load('./model.pth')
model.eval()

/var/folders/4h/8jkvbs3n7tlcqj21nw5kbbqm0000gn/T/ipykernel_29094/2282715700.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load('./model.pth')


EmotionTransformer(
  (transformer): TransformerEncoder(
    (layers): ModuleList(
      (0-7): 8 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
        )
        (linear1): Linear(in_features=768, out_features=2048, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=2048, out_features=768, bias=True)
        (norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (fc): Linear(in_features=768, out_features=7, bias=True)
)

In [22]:
# 설정
fs = 44100  # 샘플링 레이트
output_filename = "output.wav"  # 저장할 파일 이름
stop_recording = False  # 녹음 중단 플래그
seconds = 10 # 최대 녹음 시간(초)


def record_audio(): # 마이크로 음성을 녹음하는 함수.
    global stop_recording, audio_data
    print("녹음 시작. 'stop'을 입력하면 녹음을 멈춥니다.")
    audio_data = sd.rec(int(60 * fs), samplerate=fs, channels=1, dtype='int16')  # 최대 60초 녹음
    while not stop_recording:
        sd.sleep(100)  # 짧은 대기(0.1초)
    sd.stop()  # 녹음 중단
    print("녹음 중단 중...")


def wait_for_stop(): # 사용자가 'stop'을 입력할 때까지 대기.
    global stop_recording
    while not stop_recording:
        command = input("입력: ")
        if command.strip().lower() == "stop":
            stop_recording = True


# 스레드 생성 및 실행
recording_thread = threading.Thread(target=record_audio)
input_thread = threading.Thread(target=wait_for_stop)

recording_thread.start()
input_thread.start()

recording_thread.join()
input_thread.join()

# 녹음 데이터를 파일로 저장
write(output_filename, fs, audio_data[:fs * seconds])
print(f"'{output_filename}' 파일로 저장되었습니다.")


녹음 시작. 'stop'을 입력하면 녹음을 멈춥니다.
녹음 중단 중...
'output.wav' 파일로 저장되었습니다.


In [17]:
# test_audio_file = ["./output.wav"]

test_audio_file = ["./dataset/015.감성 및 발화 스타일별 음성합성 데이터/01.데이터/2.Validation/원천데이터/1.감정/1.기쁨/0029_G2A4E1S0C0_KJE/0029_G2A4E1S0C0_KJE_001970.wav", "./dataset/015.감성 및 발화 스타일별 음성합성 데이터/01.데이터/2.Validation/원천데이터/1.감정/2.슬픔/0033_G2A3E2S0C0_KMA/0033_G2A3E2S0C0_KMA_000020.wav", "./dataset/015.감성 및 발화 스타일별 음성합성 데이터/01.데이터/2.Validation/원천데이터/1.감정/3.분노/0018_G2A3E3S0C0_JBR/0018_G2A3E3S0C0_JBR_000019.wav", "./dataset/015.감성 및 발화 스타일별 음성합성 데이터/01.데이터/2.Validation/원천데이터/1.감정/4.불안/0012_G1A2E4S0C0_CHY/0012_G1A2E4S0C0_CHY_000011.wav", "./dataset/015.감성 및 발화 스타일별 음성합성 데이터/01.데이터/2.Validation/원천데이터/1.감정/5.상처/0005_G1A3E5S0C0_LJB/0005_G1A3E5S0C0_LJB_000014.wav", "./dataset/015.감성 및 발화 스타일별 음성합성 데이터/01.데이터/2.Validation/원천데이터/1.감정/6.당황/0020_G2A4E6S0C0_HGW/0020_G2A4E6S0C0_HGW_000009.wav", "./dataset/015.감성 및 발화 스타일별 음성합성 데이터/01.데이터/2.Validation/원천데이터/1.감정/7.중립/0044_G2A5E7S0C0_KTH/0044_G2A5E7S0C0_KTH_000012.wav"]

with open("./label_encoder.pkl", "rb") as f:
    label_encoder = pickle.load(f)

# Feature extractor 설정
feature_extractor = HuBERTFeatureExtractor()

# 테스트 데이터셋 및 DataLoader 생성
test_dataset = EmotionDataset(test_audio_file, feature_extractor)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False, collate_fn=collate_fn)

# 감정 예측
predicted_emotions = predict_emotion(test_dataloader, model, label_encoder)

# 결과 출력
for audio_file, emotion in zip(test_audio_file, predicted_emotions):
    print(f"Audio File: {os.path.basename(audio_file)} -> Predicted Emotion: {emotion}")

Audio File: output.wav -> Predicted Emotion: 기쁨


In [13]:
recommendations = get_recommendations(emotion, 3)

if isinstance(recommendations, str):
    print(recommendations)  # 에러 메시지 출력
else:
    print(f"'{emotion}'에 맞는 추천 콘텐츠:")
    for idx, movie in enumerate(recommendations, 1):
        print(f"\n{idx}. 제목: {movie['title']}")
        print(f"   개봉일: {movie['release_date']}")
        print(f"   평점: {movie['vote_average']}")
        print(f"   줄거리: {movie['overview']}")

'기쁨'에 맞는 추천 콘텐츠:

1. 제목: 수퍼 소닉 3
   개봉일: 2024-12-19
   평점: 7.628
   줄거리: 너클즈, 테일즈와 함께 평화로운 일상을 보내던 초특급 히어로 소닉. 연구 시설에 50년간 잠들어 있던 사상 최강의 비밀 병기 "섀도우"가 탈주하자, 세계 수호 통합 부대(약칭 세.수.통)에 의해 극비 소집된다. 소중한 것을 잃은 분노와 복수심에 불타는 섀도우는 소닉의 초고속 스피드와 너클즈의 최강 펀치를 단 단숨에 제압해버린다. 세상을 지배하려는 닥터 로보트닉과 그의 할아버지 제럴드 박사는 섀도우의 엄청난 힘 카오스 에너지를 이용해 인류를 정복하려고 하는데…

2. 제목: 모아나 2
   개봉일: 2024-11-21
   평점: 7.0
   줄거리: 바다를 누볐던 선조들에게서 예기치 못한 부름을 받은 모아나가 마우이와 다시 만나 새로운 선원들과 함께 오랫동안 잊혀진 멀고 위험한 바다 너머로 떠나는 특별한 모험을 담은 이야기

3. 제목: 레드 원
   개봉일: 2024-10-31
   평점: 7.066
   줄거리: 크리스마스 D-1, 철통같은 보안을 뚫고 코드명 '레드 원' 산타클로스가 납치되고 크리스마스가 사라질 위기에 처했다! '레드 원'을 찾기 위해 사령관 '칼럼 드리프트'는 산타클로스 따위는 없다고 믿는 현상금 사냥꾼 '잭 오말리'와 협력하기로 한다. 시작부터 삐그덕 거리는 이들 앞에 크리스마스의 존재를 위협하는 위험천만한 적들이 나타나는데…
